In [24]:
import os
import shutil

def move_images_without_annotations(source_folder, destination_folder):
    # Créer le dossier de destination s'il n'existe pas
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Parcourir les fichiers du dossier source
    for filename in os.listdir(source_folder):
        # Vérifier si le fichier est une image
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(source_folder, filename)
            # Vérifier si une annotation existe pour l'image
            annotation_path = os.path.splitext(image_path)[0] + ".txt"
            if not os.path.exists(annotation_path):
                # Déplacer l'image vers le dossier de destination
                destination_path = os.path.join(destination_folder, filename)
                shutil.copy(image_path, destination_path)
                # print(f"{filename} déplacé vers {destination_folder}")

# Chemins des dossiers source et destination
source_folder = r"C:\Users\yaoko\Documents\projects\Automotive-license-plate-recognition\data\images\cars\fr"
destination_folder = r"C:\Users\yaoko\Documents\projects\Automotive-license-plate-recognition\data\images\cars\fr_unlabeled"

# Appel de la fonction pour déplacer les images
move_images_without_annotations(source_folder, destination_folder)

In [26]:
import os
import shutil

def copy_annotations(destination_folder, source_folder):
    # Créer le dossier source s'il n'existe pas
    if not os.path.exists(source_folder):
        os.makedirs(source_folder)

    # Parcourir les fichiers du dossier de destination
    for filename in os.listdir(destination_folder):
        # Vérifier si le fichier est une annotation
        if filename.lower().endswith('.txt'):
            annotation_path = os.path.join(destination_folder, filename)
            # Copier l'annotation vers le dossier source
            source_annotation_path = os.path.join(source_folder, filename)
            shutil.copy(annotation_path, source_annotation_path)
            print(f"Annotation {filename} copiée vers {source_folder}")

# Appel de la fonction pour copier les annotations
copy_annotations(destination_folder, source_folder)

In [ ]:
import cv2
import numpy as np
import os


def rotate_image(image_path, angle):
    # Charger l'image
    image = cv2.imread(image_path)
    # Obtenir les dimensions de l'image
    height, width = image.shape[:2]
    print("image", height, width)
    # Calculer le centre de l'image
    center = (width / 2, height / 2)
    # Effectuer la rotation de l'image
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def rotate_annotation(annotation_path, image_width, image_height, angle):
    with open(annotation_path, "r") as f:
        lines = f.readlines()
    # Charger les coordonnées de l'annotation YOLO
    annotation = [float(x) for x in lines[0].strip().split()]
    print("annotation", annotation)
    # Obtenir les coordonnées du centre de l'annotation
    x_center, y_center, bbox_width, bbox_height = annotation[1:]
    # Calculer les nouveaux coordonnées après rotation
    radians = np.radians(angle)
    cos_theta = np.cos(radians)
    sin_theta = np.sin(radians)
    new_x_center = x_center * cos_theta + y_center * sin_theta
    new_y_center = -x_center * sin_theta + y_center * cos_theta
    new_bbox_width = bbox_width
    new_bbox_height = bbox_height
    # Mettre à jour les coordonnées de l'annotation YOLO après rotation
    annotation[0] = 0
    annotation[1] = new_x_center
    annotation[2] = new_y_center
    annotation[3] = new_bbox_width
    annotation[4] = new_bbox_height
    return annotation


def save_rotated_data(image_path, annotation_path, save_dir, angle):
    # Effectuer la rotation de l'image
    rotated_image = rotate_image(image_path, angle)
    # Charger les dimensions de l'image originale
    image_height, image_width = rotated_image.shape[:2]
    print("rotated_image", image_height, image_width)
    # Effectuer la rotation de l'annotation
    rotated_annotation = rotate_annotation(annotation_path, image_width, image_height, angle)
    print("rotated_annotation", rotated_annotation)
    # Enregistrer l'image tournée
    rotated_image_path = os.path.join(save_dir, os.path.basename(image_path))
    cv2.imwrite(rotated_image_path, rotated_image)
    # Enregistrer l'annotation tournée
    rotated_annotation_path = os.path.join(save_dir, os.path.basename(annotation_path))
    with open(rotated_annotation_path, "w") as f:
        f.write(" ".join(str(x) for x in rotated_annotation))
    return rotated_image_path, rotated_annotation_path


# Exemple d'utilisation :
image_folder = r"C:\Users\yaoko\Documents\projects\Automotive-license-plate-recognition\data\images\cars\fr"
image_name = "24474769.jpg"
image_path = os.path.join(image_folder, image_name)
annotation_path = os.path.join(image_folder, os.path.splitext(image_name)[0] + ".txt")
save_dir = r"C:\Users\yaoko\Documents\projects\Automotive-license-plate-recognition\data\images\cars\fr_aug"
angle = 10  # Angle de rotation en degrés
rotated_image_path, rotated_annotation_path = save_rotated_data(
    image_path, annotation_path, save_dir, angle
)
print("Image rotée sauvegardée à :", rotated_image_path)
print("Annotation rotée sauvegardée à :", rotated_annotation_path)

In [ ]:
from torch.utils.data import DataLoader
from customDataset import CustomDataset
image_folder = r"C:\Users\yaoko\Documents\projects\Automotive-license-plate-recognition\data\images\cars\fr_aug"
annotation_folder = image_folder
batch_size = 10
image_size = 416

dataset = CustomDataset(image_folder, annotation_folder, image_size=image_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
len(dataloader), len(dataset)

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# Parcourir le DataLoader pour obtenir les 6 premiers éléments
num_images = 6
images_to_display = []
annotations_to_display = []
for images, annotations in dataloader:
    for image, annotation in zip(images, annotations):
        images_to_display.append(image)
        annotations_to_display.append(annotation)
        if len(images_to_display) >= num_images:
            break
    if len(images_to_display) >= num_images:
        break

# Affichage des images
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for i, ax in enumerate(axes.flat):
    if i == len(images_to_display):
        break
    image = TF.to_pil_image(images_to_display[i])
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    ax.set_title(f'Image {i+1}')
    annotation = annotations_to_display[i]
    x_min, y_min, x_max, y_max = annotation.tolist()
    rect = plt.Rectangle(
        (x_min, y_min), x_max - x_min, y_max - y_min, edgecolor="r", facecolor="none"
    )
    ax.add_patch(rect)

plt.tight_layout()
plt.show()

In [ ]:
annotation